In [31]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import sys
sys.path.append('..')
sys.path.append('../src')
import os
from os.path import join as oj
import numpy as np
from copy import deepcopy
import pandas as pd
import data
# import models
from math import ceil
# from style import *
import dvu
import pickle as pkl
from tqdm import tqdm
# from style import *
import neural_networks
import viz
from viz import *
import config
import outcomes
outcome_def = 'successful_full'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# setup defn

In [7]:
# load model
dfs, feat_names = data.load_dfs_for_lstm(dsets=['clath_aux_dynamin'], splits=['train', 'test'],
                                         meta=['cell_num', 'Y_sig_mean', 'Y_sig_mean_normalized'], normalize=False)

100%|██████████| 1/1 [01:05<00:00, 65.04s/it]


In [13]:
df = pd.concat([dfs[(k, s)] for (k, s) in dfs])

In [14]:
# df = data.get_data(use_processed=True) #, dset='orig') # all_data=True
# df = data.get_data(dset='clath_aux_dynamin')
df = outcomes.add_aux_dyn_outcome(df)
# dfv = df[df['valid'] == 1] # exclude test cells, short/long tracks, hotspots

p = 1
results = pkl.load(open(config.FINAL_MODEL, 'rb'))
dnn = neural_networks.neural_net_sklearn(D_in=40, H=20, p=p-1, arch='lstm')
dnn.model.load_state_dict(results['model_state_dict'])

<All keys matched successfully>

In [26]:
df['preds'] = dnn.predict(df[feat_names[:1]])
df = df.sort_values(by='preds')
x = df[feat_names[:1]]
y = df['successful_full'].values
preds = dnn.predict(x)
n = df.shape[0]

# viz errs

In [37]:
# for figure
y = df[outcome_def].astype(bool)
p = df['preds'].values > 0

tp = y & p
tn = ~y & ~p
fp = ~y & p
fn = y & ~p

ks = {
    'tp': tp,
    'tn': tn,
    'fp': fn,
    'fn': fn
}
print('tp', tp.sum(), 'tn', tn.sum(), 'fp', fp.sum(), 'fn', fn.sum())


for k in ks:
    idxs = ks[k]
    num_tot = idxs.sum()
    print(k, idxs.sum())

    R = 5
    C = 10
    MULT = R * C
    for fig_num in tqdm(range(num_tot // MULT)):
        

        fig = plt.figure(figsize=(C * 3, R * 2.5), facecolor='w')
        d = df[idxs].iloc[fig_num * MULT: (fig_num + 1) * MULT]

        if k == 'fn':
            clathmax = 3500
            auxmax = 2000
        else:
            clathmax = 16000
            auxmax = 3500
    
        viz.plot_curves(d, R=R, C=C, hline=False,
                xlim_constant=True,
                ylim_constant=True,
                legend=True,
                extra_key='Z_extended',
                extra_key_label='Dynamin',
                #xlim=(0, 55),
                ylim_cla=(-500, clathmax),
                ylim_aux=(-500, auxmax),
                ylim_dyn=(-500, auxmax),
                yticks=[0, 2000],
                background=True,
                show_track_pid=True,
                #yticklabels=["$0$", "$10^4$"],                
                fig=fig
        )


        plt.savefig(f'{k}_{fig_num}.pdf')
#         plt.show()
        fig.clear()
        plt.close(fig)

  0%|          | 0/151 [00:00<?, ?it/s]

tp 7583 tn 16694 fp 3369 fn 1428
tp 7583


  0%|          | 0/333 [00:00<?, ?it/s]

tn 16694


  0%|          | 0/28 [00:00<?, ?it/s]

fp 1428


  0%|          | 0/28 [00:00<?, ?it/s]

fn 1428


100%|██████████| 28/28 [11:53<00:00, 25.49s/it]


# save out ids

In [36]:
for k in ks:
    ids = df[ks[k]].pid
    np.savetxt(k + '.csv', ids, fmt='%d', delimiter=',')